# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from scipy.optimize import *
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *
from stlab.devices.RS_SGS100A import RS_SGS100A
from stlab.devices.Keysight_N5183B import Keysight_N5183B

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice import config, RR_1_IF,RO_lo, readout_len, Q1_lo, Q1_IF, sat_Q1_len, gauss_len, R1_RS, Q1_RS
print(readout_len)

# Edit configuration parameters

**To do**

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.2.20::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(-15)
RR.setCWfrequency(RO_lo)
RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
RR.IQon()
RR.write(':SOURce:IQ:IMPairment:STATe ON')

#LO for downconversion
MXG = Keysight_N5183B(addr='TCPIP::192.168.1.91::INSTR',reset=True,verb=True)
MXG.RFon()
MXG.setCWpower(19)
MXG.setCWfrequency(RO_lo)
MXG.INTref()

QDrive = RS_SGS100A("TCPIP::169.254.50.124::INSTR", reset=True,verb=True) 
QDrive.EXTref()
QDrive.RFon()
QDrive.setCWpower(2.5)
QDrive.setCWfrequency(Q1_lo)
QDrive.write(':SOURce:IQ:IMPairment:LEAKage:I ' + Q1_RS[0])
QDrive.write('SOURce:IQ:IMPairment:LEAKage:Q ' + Q1_RS[1])
QDrive.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + Q1_RS[2])
QDrive.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + Q1_RS[3])
QDrive.IQon()
QDrive.write(':SOURce:IQ:IMPairment:STATe ON')

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# QUA Program

In [ ]:
def State_Discrimination(n_max, a_vec, b, meas_max): 
  
    with program() as state_discrimination:
        ##############################
        # declare real-time variables:
        ##############################

        n = declare(int)
        meas_n = declare(int)
        a = declare(fixed)

        A = declare(fixed)
        B = declare(fixed)
        iA = declare(fixed)
        iB = declare(fixed)
        Re = declare(fixed)
        Im = declare(fixed)

        Re_st = declare_stream()
        Im_st = declare_stream()

        with for_(meas_n, 0, meas_n < meas_max, meas_n+1):
            with for_(n, 0, n < n_max, n + 1):
                with for_each_(a, a_vec):

                    wait(int(2500/4),"RR_1","Q1")
                    play("pi"*amp(a), "Q1", duration = 14)
                    #wait(tau, "Q1", "RR_1") 
                    align("Q1", "RR_1")
                    measure("readout"*amp(b), "RR_1", None, demod.full("integW_cos", A, "out1"),
                                                                demod.full("integW_sin",B,"out2"),
                                                                demod.full("integW_sin", iA, "out1"),
                                                                demod.full("integW_cos",iB,"out2"))

                    assign(Re, A + B)       
                    assign(Im, iA - iB)
                    save(Re, Re_st)
                    save(Im, Im_st)              

        with stream_processing():
            Re_st.buffer(meas_max,n_max, 2).map(FUNCTIONS.average(1)).save("Re")
            Im_st.buffer(meas_max,n_max, 2).map(FUNCTIONS.average(1)).save("Im")

    job = qm.execute(state_discrimination, duration_limit=0, data_limit=0)

    res_handles= job.result_handles
    res_handles.wait_for_all_values()

    Re_handle = res_handles.get("Re")
    Im_handle = res_handles.get("Im")

    Re_s = Re_handle.fetch_all()
    Im_s = Im_handle.fetch_all()

    return Re_s, Im_s

# Define Parameters

In [ ]:
n_max = 500
meas_max = 10000

a_vec = [0,1.4]

#Readout pulse amplitude
b = 0.24

In [ ]:
Re, Im = State_Discrimination(n_max, a_vec, b, meas_max)

# Fetch data and save to .dat

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'Q1_State_Discrimination'

data_Re = Re
data_Im = Im

data = np.asarray([(np.array(a_vec)).reshape(len(a_vec),1)+Re-Re,data_Re,data_Im])

for i,amps in enumerate(a_vec):
    print(i,amps)
    data_dict={'Qubit Pulse Amplitude':data[0][i],
           'Real':data[1][i],
           'Imaginary':data[2][i],
              }
           
    if i==0:
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        
        myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True)
        
        output = new_stdout.getvalue()
        sys.stdout = old_stdout
        print(output)
        M_ind = output.find("Measurement Name")
        M_name = output[M_ind+len('Measurement Name:  '):-1]
    stlab.savedict(myfile,data_dict)
    stlab.metagen.fromarrays(myfile,(np.array(a_vec))[0:i+1],[0,0],xtitle='Qubit Pulse Amplitude',ytitle='Nothing',colnames=list(data_dict))

# Processing/Plotting

In [ ]:
plt.figure(figsize=(8,6),dpi = 100)
plt.scatter(Re[0,:],Im[0,:],label = 'g', s=1)
plt.scatter(Re[1,:],Im[1,:],label = 'e',s =1)
plt.axhline(0,ls='--', c= 'black')
plt.axvline(0,ls='--', c= 'black')
plt.xlabel('I', fontsize = 14)
plt.ylabel('Q', fontsize = 14)
plt.xticks(fontsize=14); plt.yticks(fontsize=14);

g_max = np.max(np.abs(Re[0,:]+1j*Im[0,:]))
e_min = np.min(np.abs(Re[1,:]+1j*Im[1,:]))
th = (g_max+e_min)/2
#print(th)

s = np.linspace(0,2*np.pi,300)
c_th = th*np.exp(1j*s)
plt.plot(np.real(c_th),np.imag(c_th), ls ='-.', color ='red', label = 'threshold');
plt.axis('equal');
plt.legend(fontsize=14, loc = 'best');

# Save this file and configuration file to measurement folder

In [ ]:
#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_BMDevice.py')
save_config_path = os.path.join(meas_path, 'Configuration_BMDevice.py')

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);

#### Some plotting code that might be useful 

In [ ]:
fig = plt.figure(figsize=(14,14),dpi=100) #,constrained_layout=True)
gs = GridSpec(3, 3, figure=fig)
ax1 = fig.add_subplot(gs[0:2,0:2])
ax2 = fig.add_subplot(gs[2,0:2])
ax3 = fig.add_subplot(gs[:2,2])


#.axhline(0,ls='-', c= 'black',lw=0.5)
#plt.axvline(0,ls='-', c= 'black',lw=0.5)
ax1.scatter(Re,Im,label = 'g', s = 1,alpha=1)
ax1.scatter(Re1,Im1, label = 'e', s= 1,alpha=1)
ax1.set_xlabel('I', fontsize = 12)
ax1.set_ylabel('Q', fontsize = 12)
ax1.tick_params(labelsize = 12)
ax1.legend(fontsize = 14)

ax2.hist(Re,bins=100,histtype = 'step', label = 'g');
ax2.hist(Re1,bins=100,histtype = 'step',label = 'e');
ax2.legend(fontsize = 14)

ax3.hist(Im,bins=100,histtype = 'step', orientation ='horizontal', label = 'g');
ax3.hist(Im1,bins=100,histtype = 'step', orientation = 'horizontal', label = 'e');
ax3.legend(fontsize = 14)

#print(np.abs(Re+1j*Im))
#print(np.abs(Re1+1j*Im1))